## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jiuquan,CN,39.7432,98.5174,62.83,30,83,0.51,broken clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,36.55,66,100,18.50,overcast clouds
2,2,Killybegs,IE,54.6333,-8.4500,59.45,83,87,17.02,overcast clouds
3,3,Thompson,CA,55.7435,-97.8558,47.79,56,97,2.28,overcast clouds
4,4,Kaitangata,NZ,-46.2817,169.8464,38.50,91,15,7.09,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                       (city_data_df['Max Temp'] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Georgetown,MY,5.4112,100.3354,80.89,88,98,6.44,overcast clouds
10,10,Butaritari,KI,3.0707,172.7902,82.72,69,78,15.93,broken clouds
11,11,Caravelas,BR,-17.7125,-39.2481,82.18,64,5,12.19,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,77.65,74,33,23.73,scattered clouds
23,23,Salvador,SV,13.8333,-88.9167,81.05,73,97,1.57,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
658,658,Saint-Georges,GD,12.0564,-61.7485,84.96,70,24,11.97,few clouds
659,659,Mahibadhoo,MV,3.7833,72.9667,83.50,76,9,13.91,clear sky
661,661,Dakar,SN,14.6937,-17.4441,83.41,77,95,10.25,overcast clouds
663,663,Daru,PG,-9.0763,143.2092,76.71,90,88,13.98,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Georgetown,MY,80.89,overcast clouds,5.4112,100.3354,
10,Butaritari,KI,82.72,broken clouds,3.0707,172.7902,
11,Caravelas,BR,82.18,clear sky,-17.7125,-39.2481,
16,Atuona,PF,77.65,scattered clouds,-9.8000,-139.0333,
23,Salvador,SV,81.05,overcast clouds,13.8333,-88.9167,
24,Alofi,NU,76.06,few clouds,-19.0595,-169.9187,
29,Nhulunbuy,AU,79.21,scattered clouds,-12.2333,136.7667,
30,Hilo,US,78.28,light rain,19.7297,-155.0900,
33,Port Hedland,AU,79.97,overcast clouds,-20.3167,118.5667,
47,Kalmunai,LK,81.91,scattered clouds,7.4167,81.8167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = ('WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))